In [2]:
import cv2
import numpy as np

#Load our image
image = cv2.imread("images/test.png")
cv2.imshow("Original image",image)
cv2.waitKey()

#Create a black image with same dimensions as our loaded image
blank_image = np.zeros((image.shape[0],image.shape[1],3))

#create a copy of our original image
original_image=image

#grayscale our image
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

#find the edges
edged = cv2.Canny(gray,50,200)
cv2.imshow("Canny Edges",edged)
cv2.waitKey()

#Find contours and print how many were found
_,contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of contours found: "+str(len(contours)))

#Draw all contours
cv2.drawContours(image,contours,-1,(0,255,0),3)
cv2.imshow("All contours",image)
cv2.waitKey()

#Draw all contours over blank image
cv2.drawContours(blank_image,contours,-1,(0,255,0),3)
cv2.imshow("All contours over blank image",blank_image)
cv2.waitKey()

cv2.destroyAllWindows()

Number of contours found: 5


Let's now sort by area

In [3]:
import cv2
import numpy as np

#function we'll  use to display contour area

def get_contour_areas(contours):
    #return the areas of all contours as list
    all_areas=[]
    for cnt in contours:
        area=cv2.contourArea(cnt)
        all_areas.append(area)
    return all_areas

#Load our image
image = cv2.imread("images/test.png")
original_image=image

#Let's print the areas of the contours before sorting
print("Contor areas before sorting")
print(get_contour_areas(contours))

#Sort contours large to small
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
#sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]

print("Contor areas after sorting")
print(get_contour_areas(sorted_contours))

#Iterate over our contours and draw one at a time
for c in sorted_contours:
    cv2.drawContours(original_image,[c],-1,(255,0,0),3)
    cv2.waitKey()
    cv2.imshow("Contours by area",original_image)
    
cv2.waitKey()
cv2.destroyAllWindows()

Contor areas before sorting
[2214.0, 2748.0, 6435.0, 12405.0, 7716.0]
Contor areas after sorting
[12405.0, 7716.0, 6435.0, 2748.0, 2214.0]


Sorting contours from left to right

In [6]:
#functions we'll use for sorting by position

def x_cord_contour(contours):
    #Return the X cordinate for contour centroid
    if cv2.contourArea(contours)>10:
        M = cv2.moments(contours) #returns the center point
        return(int(M['m10']/M['m00']))
    
def label_contour_center(image,c):
    #places a red circle on the centers of contours
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    #draw the contour number on the image
    cv2.circle(image,(cx,cy),10,(0,0,255),-1)
    return image

#Load our image
image = cv2.imread("images/test.png")
original_image=image.copy()

#compute center of mass of centroids and draw them on our image
for(i,c) in enumerate(contours):
    orig=label_contour_center(image,c)
    
cv2.imshow("Contour centers",image)
cv2.waitKey()

#sort by left to right using our x_cord_contour function
contours_left_to_right = sorted(contours, key = x_cord_contour,reverse=False)

#Labeling contours left to right
for(i,c) in enumerate(contours_left_to_right):
    cv2.drawContours(original_image,[c],-1,(0,0,255),3)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_image, str(i+1), (cx,cy), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow("Left to right contour", original_image)
    cv2.waitKey()
    (x,y,w,h)=cv2.boundingRect(c)
    #Let's now crop each countour and save these images
    cropped_contour=original_image[y:y+h,x:x+w]
    image_name="output_shape_number_"+str(i+1)+".jpg"
    print(image_name)
    cv2.imwrite(image_name,cropped_contour)
    
cv2.destroyAllWindows()

output_shape_number_1.jpg
output_shape_number_2.jpg
output_shape_number_3.jpg
output_shape_number_4.jpg
output_shape_number_5.jpg
